# Opioid Addiction Project
## Notebook 02: Exploratory Data Analysis

This notebook explores the data set.

### W210, Capstone
Summer 2019

Team:  Cameron Kennedy, Aditi Khullar, Rachel Kramer, Sharad Varadarajan

# 0. Load Libraries and Set Global Variables
This analysis is performed in the cells below.

In [1]:
#Import Required Libraries
import numpy as np
import pandas as pd
from statsmodels.stats.weightstats import DescrStatsW
from collections import OrderedDict
from operator import itemgetter
import pickle

#Custom data prep function used in both training and prediction 
import OpioidDataPrep as odp

#Set initial parameter(s)
pd.set_option('display.max_rows', 200)
pd.options.display.max_columns = 150
dataDir = './data/'

print('Pandas Version', pd.__version__)

Pandas Version 0.24.2


# 1. Load Data

This step loads the data from the file `misuse.pickle.zip`.

In [2]:
#Load Data
df = pd.read_pickle(dataDir+'misuse.pickle.zip')
df

,AALTMDE,ABODALC,ABODCOC,ABODHER,ABODMRJ,ABPYILANAL,ABPYILLALC,ABUPOSHAL,ABUPOSINH,ABUPOSMTH,ABUPOSPNR,ABUPOSSED,ABUPOSSTM,ABUPOSTRQ,ABUSEALC,ABUSECOC,ABUSEHER,ABUSEMRJ,ABUSEPYHAL,ABUSEPYIEM,ABUSEPYILL,ABUSEPYINH,ABUSEPYMTH,ABUSEPYPNR,ABUSEPYPSY,ABUSEPYSED,ABUSEPYSTM,ABUSEPYTRQ,ACOUNMDE,ACTD2001,ACTD7590,ACTD9001,ACTDEVER,ACTDPRIV,ACTDVIET,ADCOUNS,ADDPDISC,ADDPLSIN,ADDPPROB,ADDPR2WK,ADDPREV,ADDSCEV,ADDSLSIN,ADFAMDOC,ADHERBAL,ADLOSEV,ADLSI2WK,ADNURSE,ADOCMDE,ADOTHDOC,ADOTHHLP,ADOTHMHP,ADPB2WK,ADPBAGE,ADPBDLYA,ADPBINTF,ADPBNUM,ADPBRMBR,ADPSDAYS,ADPSHMGT,ADPSRELS,ADPSSOC,ADPSWORK,ADPSYCH,ADPSYMD,ADRELIG,ADRX12MO,ADRXHLP,ADRXNOW,ADSEEDOC,ADSMMDEA,ADSOCWRK,ADTMTHLP,ADTMTNOW,ADWRAGE,...,YUIHFEAR,YUIHFITE,YUIHFMLY,YUIHFRND,YUIHOTPP,YUIHSCHL,YUIHSOR,YUIHSUIC,YUIHTPN2,YUIHTPYR,YUJVDTN2,YUJVDTON,YUJVDTYR,YUMHANGR,YUMHBKRU,YUMHCRN2,YUMHCRYR,YUMHDEPR,YUMHEATP,YUMHFEAR,YUMHFITE,YUMHFMLY,YUMHFRND,YUMHOTPP,YUMHSCHL,YUMHSOR,YUMHSUIC,YURSANGR,YURSBKRU,YURSDEPR,YURSEATP,YURSFEAR,YURSFITE,YURSFMLY,YURSFRND,YURSIDN2,YURSIDYR,YURSOTPP,YURSSCHL,YURSSOR,YURSSUIC,YUSCEMYR,YUSCPGYR,YUSWANGR,YUSWBKRU,YUSWDEPR,YUSWEATP,YUSWFEAR,YUSWFITE,YUSWFMLY,YUSWFRND,YUSWOTPP,YUSWSCHL,YUSWSCYR,YUSWSOR,YUSWSUIC,YUTPANGR,YUTPBKRU,YUTPDEPR,YUTPEATP,YUTPFEAR,YUTPFITE,YUTPFMLY,YUTPFRND,YUTPOTPP,YUTPSCHL,YUTPSOR,YUTPSTN2,YUTPSTYR,YUTPSUIC,ZALEPDAPYU,ZOHYANYYR2,ZOLPPDAPYU,ZOLPPDPYMU,MISUSE
19,NaN,0,0,0,0,0,0,91.0,91.0,91.0,91.0,91.0,91.0,91.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,NaN,99,99,99,99,999,99,99,9999,99,999,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,999,...,99,99,99,99,99,99,99,99,999,2,999,2,99,99,99,999,2,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,999,2,99,99,99,99,2,2,99,99,99,99,99,99,99,99,99,99,2,99,99,99,99,99,99,99,99,99,99,99,99,99,999,2,99,0,0,0,0,0
20,0.0,1,0,0,0,0,1,93.0,91.0,91.0,91.0,91.0,91.0,91.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,99,99,99,99,99,99,99,1,1,1,1,1,99,99,99,99,99,99,99,0.0,99,99,99,2,15,2,2,4,2,999,99,99,99,99,99,99,99,2,99,99,2,1,99,99,99,13,...,99,99,99,99,99,99,99,99,999,99,999,99,99,99,99,999,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,999,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,999,99,99,0,0,0,0,0
21,NaN,0,0,0,0,0,0,91.0,93.0,91.0,91.0,91.0,91.0,91.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,99,99,99,99,99,99,99,99,99,99,99,2,2,99,99,99,2,99,99,NaN,99,99,99,99,999,99,99,9999,99,999,99,99,99,99,99,99,99,99,99,99,99,98,99,99,99,999,...,99,99,99,99,99,99,99,99,999,99,999,99,99,99,99,999,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,999,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,999,99,99,0,0,0,0,0
23,NaN,0,0,0,0,0,0,91.0,91.0,91.0,93.0,91.0,91.0,91.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,99,99,99,99,99,99,99,99,99,99,99,2,2,99,99,99,2,99,99,NaN,99,99,99,99,999,99,99,9999,99,999,99,99,99,99,99,99,99,99,99,99,99,98,99,99,99,999,...,99,99,99,99,99,99,99,99,999,99,999,99,99,99,99,999,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,999,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,999,99,99,0,0,0,0,0
33,NaN,0,0,0,0,0,0,91.0,91.0,91.0,91.0,91.0,91.0,91.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,99,99,99,99,99,99,99,99,99,99,99,2,2,99,99,99,2,99,99,NaN,99,99,99,99,999,99,99,9999,99,999,99,99,99,99,99,99,99,99,99,99,99,98,99,99,99,999,...,99,99,99,99,99,99,99,99,999,99,999,99,99,99,99,999,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,999,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,999,99,99,0,0,0,0,0
34,NaN,0,0,0,0,0,0,91.0,91.0,91.0,0.0,91.0,0.0,91.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,99,99,99,99,99,99,99,99,99,99,99,2,2,99,99,99,2,99,99,NaN,99,99,99,99,999,99,99,9999,99,999,99,99,99,99,99,99,99,99,99,99,99,98,99,99,99,999,...,99,99,99,99,99,99,99,99,999,99,999,99,99,99,99,999,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,999,99,99,99,99,99,99,99,99,99,99,99

# 2. Variation of Misuse by Feature

Here we seek to find features that have a high standard deviation in their prevalence of opioid misuse.  What's critical here is that we weight this standard deviation by the N-size. Why? Take, for example, a case where we have a high standard deviation, but a very low N-Size in one category. In this case, although there's a large difference, it won't be that useful because there aren't many individuals who fall in to that category.  Instead, we need to find variables where there's a large difference in misuse and that have large n-sizes.

In [3]:
#Helper functions
def calcM1Pct(dfIn, colIn):
    #Function to create a data frame of misuse percentages and N-size
    s0 = df[df['MISUSE']==0][colIn].value_counts()
    s1 = df[df['MISUSE']==1][colIn].value_counts()
    
    dfOut = pd.concat([s0, s1], axis=1)
    dfOut.columns = ['M0', 'M1']
    dfOut['MTot'] = dfOut['M0'] + dfOut['M1']
    dfOut['M1Pct'] = dfOut['M1'] / dfOut['MTot']
    
    dfOut.dropna(inplace=True)
    
    return dfOut

def calcWeightedStd(dfIn, colIn):
    #Funciton to calculate weighted standard deviation (and regular stdev if desired)
    dfM1Pct = calcM1Pct(dfIn, colIn)
    weighted_stats = DescrStatsW(dfM1Pct.M1Pct, weights=dfM1Pct.MTot, ddof=0)
    
    return weighted_stats.std, np.std(dfM1Pct.M1Pct) #Returns the weighted and unweighted std

In [8]:
#Output of data frame misuse percengates and n-sizes
calcM1Pct(df, 'ADDPREV')

,M0,M1,MTot,M1Pct
1,13705,3656.0,17361.0,0.210587
2,24225,3498.0,27723.0,0.126177
94,62,24.0,86.0,0.279070
97,139,34.0,173.0,0.196532
99,6520,1471.0,7991.0,0.184082


In [5]:
#Examples to illustate how this works
'''HLLOSRSN is a good example where weighting matters, there's lots of variation,
but nearly all respondents picked one answer, thus resulting in a low score.

Conversely, ABUPOSTRQ is a good example of a function that has enough variation with
higher n-sizes, pushing it to be a highly weighted function.
''';

def printWtStdev(colIn):
    print('VARIABLE:', colIn)
    print('TABLE OF MISUSE PERCENTAGE\n',calcM1Pct(df, colIn))
    print('UNWEIGHTED STANDARD DEVIATION:',calcWeightedStd(df, colIn)[1])
    print('  WEIGHTED STANDARD DEVIATION:',calcWeightedStd(df, colIn)[0])
    print()


printWtStdev('ADDPREV')

VARIABLE: ADDPREV
TABLE OF MISUSE PERCENTAGE
        M0      M1     MTot     M1Pct
1   13705  3656.0  17361.0  0.210587
2   24225  3498.0  27723.0  0.126177
94     62    24.0     86.0  0.279070
97    139    34.0    173.0  0.196532
99   6520  1471.0   7991.0  0.184082
UNWEIGHTED STANDARD DEVIATION: 0.049146429119458615
  WEIGHTED STANDARD DEVIATION: 0.039164770201125174



In [6]:
### THIS CELL TAKES A LONG TIME (20 min?) TO RUN IF runThis=True ###

'''This cell creates a dictionary of each of the columns and their weighted
standard deviation of the percentage of misuse in each of their categories.

This dictionary can then be used to help determine which features are likely
to provide strong predictors. While not a perfect technique, it's a great
start for feature engineering.

Of note, however, many of the top features should not be used, because they
are likely "downstream" from opioid misuse and thus suffer look ahead bias,
making them poor predictors in real life, even if they're strong in this data
set. Therefore, each feature must also be examined for this potential and
omitted if it is thought likely to exist.
'''

runThis=False
if runThis:
    weightedStdByColDict = {}
    for i, col in enumerate(df.columns):
        if i%25==0:
            print('Column',i,'of',len(df.columns))
        weightedStdByColDict[col] = calcWeightedStd(df, col)[0]
        
    sortedWtDict = OrderedDict(sorted(weightedStdByColDict.items(), key=itemgetter(1), reverse=True))
    
    with open(dataDir+'EDAsortedWtDict.pickle','wb') as f:
        pickle.dump(sortedWtDict,f)
    
else:
    with open(dataDir+'EDAsortedWtDict.pickle','rb') as f:
        sortedWtDict = pickle.load(f)
        
#Print dict
sortedWtDict

OrderedDict([('OXCOPDPYMU', 0.21554667814567785),
             ('PSYCHMON', 0.19070331373005475),
             ('OPINMMON', 0.18818564022860124),
             ('ILLEMMON', 0.1870337641001585),
             ('PNRNMMON', 0.18636004482464633),
             ('ILLANDALC', 0.18630631642724074),
             ('PNRNDAYPM', 0.17619012246287047),
             ('PNRLWD3SX', 0.16340745424692937),
             ('ILLMON', 0.155383768916722),
             ('IRPNRNM30FQ', 0.15008063760474144),
             ('UDPYILL', 0.1463640996477841),
             ('TRQNMREC', 0.1445925811424573),
             ('IRTRQNMREC', 0.14454088423243122),
             ('TRQRSHIGH', 0.14441135150029585),
             ('TRQNM30D', 0.14389683679560641),
             ('TRQNMINIT', 0.14383141555661433),
             ('IRTRQNMINIT', 0.14378365764675927),
             ('TXYRNDILL', 0.1437653205076837),
             ('UDPYIEM', 0.1437188961690521),
             ('TRQLLSEFT', 0.14358775388918474),
             ('TRQLNDMOR', 0.14356

In [7]:
#Example of join
omitCols = ['OX','MON','TRQ','PNR','UDP']
'|'.join(omitCols)

'OX|MON|TRQ|PNR|UDP'

In [8]:
#Filter values
omitCols = ['OX', #OXycodone
            'MON', #Month (in the past 30 days ...)
            'TRQ', #Transquilizers
            'PNR', #Pain relievers
            'UDP', 'DPY', 'DEP', #Substance dependence             
            'ILL', #Used ILLegial / ILLicit drugs
            
            #The rest are not directly related to pain killers
            'TXY', #Needed or received treatment for drugs/alc
            'MU', #Misuse
            'HALU', 'HALL', 'HAL', #Hallucinigens (sp)
            'STIM', 'STM', #Stimulants
            'MR', 'MJ', #Weed
            'COC', 'CC', #Cocaine
            'BLNT', #Blunts
            'CAD', #Alcohol + Drugs (combined usage)
            'LSD', 'PCP', 'MESC','PEYOTE','KETMINESK','PSILCY','HER',#Other hard drugs
            'INH', #Inhaled Drugs
            'AMP','AMM', #Amphetamines
            'SALVIA',
            'SED', #Sedatives
           ]
omitColsOrPipe = '|'.join(omitCols)
keys = df.columns[df.columns.str.contains('^(?:(?!'+omitColsOrPipe+').)*$')]

tempDict = {x: sortedWtDict[x] for x in keys}
filterDict = OrderedDict(sorted(tempDict.items(), key=itemgetter(1), reverse=True))
print(len(filterDict),'rows, omitting', omitColsOrPipe)
filterDict

1523 rows, omitting OX|MON|TRQ|PNR|UDP|DPY|DEP|ILL|TXY|MU|HALU|HALL|HAL|STIM|STM|MR|MJ|COC|CC|BLNT|CAD|LSD|PCP|MESC|PEYOTE|KETMINESK|PSILCY|HER|INH|AMP|AMM|SALVIA|SED


C:\Users\camke\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in less


OrderedDict([('HPDRGTALK', 0.12282761411376411),
             ('YOSELL2', 0.11115720508061938),
             ('SMIPP_U', 0.11110659088990259),
             ('DMTAMTFXY', 0.106742285620871),
             ('CONDLREC', 0.09475774836867072),
             ('SIMHBRK2', 0.09308240792363298),
             ('CRKEVER', 0.0916961158583234),
             ('ALPRPDAPYU', 0.09042261343270319),
             ('FRDMEVR2', 0.09010203280053328),
             ('SMHSUI2', 0.08596081377588587),
             ('HRDAYPWK', 0.08520868460035495),
             ('HRBSTWAY', 0.08499221286620565),
             ('AIRDUSTER', 0.0849892692474125),
             ('IRCD2YFU', 0.08477534373034389),
             ('IRCIGYFU', 0.08420766871884039),
             ('SOMHSUI', 0.08312448725428563),
             ('HRDAYPMO', 0.08247252868672565),
             ('HRSNFREC', 0.0824511801801238),
             ('HRNDLREC', 0.0818374512809819),
             ('ALCTRY', 0.08145745537682554),
             ('HR30EST', 0.08134640887759531),
 

In [9]:
#Alternatively, look at features we're including

colsToKeep = [#DEMOGRAPHICS
              'IRSEX', #(Gender):  Subtract 1 from this field to make it 0=Male, 1=Female
              'IREDUHIGHST2', #(Highest Education) - Removed in favor of EDUHIGHCAT
              'EDUHIGHCAT', #(Highest Education)
              'AGE2', #(Age): Scale and center
    
              #ALCOHOL
              'IRALCRC', #(Alcohol Recency): Straight One Hot
              'IRALCFY', #(Alcohol Frequency Past Year):  Bin
              'BNGDRKMON', #(Binge drinking, past 30 days):  No action required (already binary, 0 and 1)
              'HVYDRKMON', #(Heavy drinking, past 30 days):  No action required (already binary, 0 and 1)
              'IRALCAGE', #(First time used alcohol): Bin

              #DRUGS + ALCOHOL
              'TXYRRECVD2', #(Ever alcohol/drug treatment, past yr): No action required (already binary, 0 and 1)
              'TXEVRRCVD2', #(Ever alcohol/drug treatment, lifetime): No action required (already binary, 0 and 1)

              #'DRVINALCO2', #Not found in data
              #TOBACCO
              'IRCIGRC', #(Tobacco Recency, incl. Never):  Straight One Hot
              'CIGDLYMO', #(Tobacco 30+ consecutive days):  Straight One Hot
              'CIGAGE', #(Tobacco Use Daily):  Bin
              'PIPEVER', #(Ever smoked a pipe): Straight One Hot
              'IRCGRRC', #(Cigar recency): Straight One Hot
              'IRSMKLSSREC', #(Smokeless tobacco recency): Straight One Hot
    
              #WEED
              'IRMJRC', #(Weed recency): Straight One Hot
              'MJYRTOT', #(Weed days in past year):  Bin
              'FUMJ18', #(First used weed prior to age 18): Subtract 1
              'FUMJ21', #(First used weed prior to age 21): Subtract 1
              #'DRVINMARJ', #Not found in data
    
              #DEPRESSION
              'ADDPREV', #(Several days of depression): One Hot
              'ADDSCEV', #(Several days of discouraged about life): One Hot
    
              ##OTHER
              'BOOKED', #(Ever arrested & booked): Recode + One Hot
                      
              #OUTCOME VARIABLE    
              'MISUSE',
             ]

tempDict = {x: sortedWtDict[x] for x in colsToKeep}
filterDict = OrderedDict(sorted(tempDict.items(), key=itemgetter(1), reverse=True))
print(len(filterDict),'rows')
filterDict

25 rows


OrderedDict([('MJYRTOT', 0.12496808513807363),
             ('IRMJRC', 0.11815138834303235),
             ('FUMJ18', 0.08898698469732465),
             ('FUMJ21', 0.08281433335329229),
             ('IRALCAGE', 0.08125284104464078),
             ('IRCIGRC', 0.07871385749027747),
             ('IRCGRRC', 0.06859377463123052),
             ('TXEVRRCVD2', 0.06439127720680231),
             ('AGE2', 0.06409381240417858),
             ('TXYRRECVD2', 0.06217275120570994),
             ('BNGDRKMON', 0.06171120557469351),
             ('IRALCFY', 0.060490486285784024),
             ('IRSMKLSSREC', 0.059973373730271506),
             ('CIGAGE', 0.05618291173601904),
             ('CIGDLYMO', 0.05603095689085829),
             ('HVYDRKMON', 0.05479880313836803),
             ('BOOKED', 0.052755627335141844),
             ('IRALCRC', 0.04540993984204561),
             ('ADDPREV', 0.039164770201125174),
             ('ADDSCEV', 0.03891168604386702),
             ('PIPEVER', 0.038890773890793595),
